In [45]:
# Natural Language Toolkit: Chatbot Utilities
#
# Copyright (C) 2001-2022 NLTK Project
# Authors: Steven Bird <stevenbird1@gmail.com>
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT

# Based on an Eliza implementation by Joe Strout <joe@strout.net>,
# Jeff Epler <jepler@inetnebr.com> and Jez Higgins <jez@jezuk.co.uk>.

import random
import re
from nltk.stem import PorterStemmer

reflections = {
    "i am": "you are",
    "i was": "you were",
    "i": "you",
    "i'm": "you are",
    "i'd": "you would",
    "i've": "you have",
    "i'll": "you will",
    "my": "your",
    "you are": "I am",
    "you were": "I was",
    "you've": "I have",
    "you'll": "I will",
    "your": "my",
    "yours": "mine",
    "you": "me",
    "me": "you",
}


class Chat:
    def __init__(self, pairs, reflections={}):
            """
            Initialize the chatbot.  Pairs is a list of patterns and responses.  Each
            pattern is a regular expression matching the user's statement or question,
            e.g. r'I like (.*)'.  For each such pattern a list of possible responses
            is given, e.g. ['Why do you like %1', 'Did you ever dislike %1'].  Material
            which is matched by parenthesized sections of the patterns (e.g. .*) is mapped to
            the numbered positions in the responses, e.g. %1.

            :type pairs: list of tuple
            :param pairs: The patterns and responses
            :type reflections: dict
            :param reflections: A mapping between first and second person expressions
            :rtype: None
            """

            self._pairs = [(re.compile(x, re.IGNORECASE), y) for (x, y) in pairs]
            self._reflections = reflections
            self._regex = self._compile_reflections()


    def _compile_reflections(self):
            sorted_refl = sorted(self._reflections, key=len, reverse=True)
            return re.compile(
                r"\b({})\b".format("|".join(map(re.escape, sorted_refl))), re.IGNORECASE
            )

    def _substitute(self, str):
            """
            Substitute words in the string, according to the specified reflections,
            e.g. "I'm" -> "you are"

            :type str: str
            :param str: The string to be mapped
            :rtype: str
            """

            return self._regex.sub(
                lambda mo: self._reflections[mo.string[mo.start() : mo.end()]], str.lower()
            )

    def _wildcards(self, response, match):
            pos = response.find("%")
            while pos >= 0:
                num = int(response[pos + 1 : pos + 2])
                response = (
                    response[:pos]
                    + self._substitute(match.group(num))
                    + response[pos + 2 :]
                )
                pos = response.find("%")
            return response

    def respond(self, str):
            """
            Generate a response to the user input.

            :type str: str
            :param str: The string to be mapped
            :rtype: str
            """

            # check each pattern
            for (pattern, response) in self._pairs:
                match = pattern.match(str)

                # did the pattern match?
                if match:
                    resp = random.choice(response)  # pick a random response
                    resp = self._wildcards(resp, match)  # process wildcards

                    # fix munged punctuation at the end
                    if resp[-2:] == "?.":
                        resp = resp[:-2] + "."
                    if resp[-2:] == "??":
                        resp = resp[:-2] + "?"
                    return resp
                
    def modified_converse(self, quit="quit"):
            stemmer = PorterStemmer() #stemmer
            user_input = ""
            stemmed_user_input = [] #store stemmed user input
            
            user_log = [] # storing all the user inputs
            stem_log = set() # store type, unique words
            
            while user_input != quit:
                user_input = quit
                try:
                    user_input = input(">")
                except EOFError:
                    print(user_input)
                if user_input:
                    userWords = user_input.lower().split(' ') #tokenizing user input
                    for word in userWords:
                        user_log.append(word)
                        stemmed_user_input.append(stemmer.stem(word))
                        stem_log.add(stemmer.stem(word)) # holds the type from chat session
                        
                    while user_input[-1] in "!.":
                        user_input = user_input[:-1]
                    stemInput = ' '.join(stemmed_user_input)
                    #print("---User Entered  : ", user_input)
                    print("---Stemmed Input : ", stemInput) #prints the stemmed string entered by user
                    #print("---Type log      : ", stem_log) #prints all the type from chat session
                    print("---Total #of type: ", len(stem_log))
                    print("---# of Tokens   : ", len(user_log))
                    print(self.respond(stemInput)) #passing stemmed user input
                    stemmed_user_input.clear() #clearing stemmed input after each input

# Hold a conversation with a chatbot

# Natural Language Toolkit: Eliza
#
# Copyright (C) 2001-2022 NLTK Project
# Authors: Steven Bird <stevenbird1@gmail.com>
#          Edward Loper <edloper@gmail.com>
# URL: <https://www.nltk.org/>
# For license information, see LICENSE.TXT


# Based on an Eliza implementation by Joe Strout <joe@strout.net>,
# Jeff Epler <jepler@inetnebr.com> and Jez Higgins <mailto:jez@jezuk.co.uk>.


# a translation table used to convert things you say into things the
# computer says back, e.g. "I am" --> "you are"


# from nltk.chat.util import Chat, reflections

# a table of response pairs, where each pair consists of a
# regular expression, and a list of possible responses,
# with group-macros labelled as %1, %2.


pairs = (
    (
        r"Yeah",
        (
            "Anything else?",
           
        ),
    ),
    (
        r"I rememb (.*)",
        (
            "Do you miss that?",
            "Do you wish you were %1?",
           
        ),
    ),
    (
        r"My name is (.*)",
        (
            "Hi %1 I'm Eliza, How are you today?",
            "Nice to meet you %1, how can I help you today?",
            "Hi %1, I'm Eliza. Ask me anything",
        ),
    ),
    (
        r"What' the weather (.*)",
        (
            "%1 is not too bad.",
            "%1 is better than yesterday.",
            "%1 is 70 degree. I hope you have a good day.",
        ),
    ),
    (
        r"Thank you",
        (
            "No worries",
            "You're welcome",
            "My Pleasure",
        ),
    ),
    (
        r"What' (.*)",
        (
            "I'm not too sure about %1, I'll let you know tomorrow",
            "Perhaps you could google %1",
        ),
    ),
    (
        r"I am feel(.*)",
        (
            "Everyone feels %1. ",
            "Why are you %1?",
        ),
    ),
  (
        r"I\'m(.*)",
        (
            "Thank you for telling me. How can I help you today?",
        ),
    ),
    (
        r"What do you think about (.*)",
        (
            "%1 is okay, nothing amazing. What do you think?",
        ),
    ),
    (
        r"Do you know if (.*)",
        (
            "Hmmm, it depends",
            "I don't know if %1",
        ),
    ),


    (
        r"sorry ",
        (
            "No worries",
            "You're good",
        ),
    ),
    (
        r"Hello(.*)",
        (
            "Hello... What's your name?",
        ),
    ),
    (
        r"I think (.*)",
        (
            "Well that's good you think that way.",
            "Everyone has their own opinion.",
            
        ),
    ),
 
    (
        r"I have (.*)",
        (
            "I had %1 long time ago. What do you think about %1?",
            
        ),
    ),
    
    (
        r"Is there (.*)",
        (
            "I beleive there is %1 somewhere",
        ),
    ),


    (
        r"Do you believe in (.*)",
        (
            "Yes, I do believe in %1. ",
        ),
    ),




    (
        r"(.*)\?",
        (
            "Why do you ask that?",
            "Please consider whether you can answer your own question.",
            "Perhaps the answer lies within yourself?",
            "Why don't you tell me?",
        ),
    ),
    (
        r"bye",
        (
            "bye. Hopefully, we'll talk soon",
        ),
    ),
    (
        r"(.*)",
        (
            "Please tell me more.",
            "Let's change focus a bit... Tell me about your family.",
            "Can you elaborate on that?",
            "Why do you say that %1?",
            "I see.",
            "Very interesting.",
            "%1.",
            "I see.  And what does that tell you?",
            "How does that make you feel?",
            "How do you feel when you say that?",
        ),
    ),
)

eliza_chatbot = Chat(pairs, reflections)




def eliza_chat():
    print("Therapist\n---------")
    print("Talk to the program by typing in plain English, using normal upper-")
    print('and lower-case letters and punctuation.  Enter "quit" when done.')
    print("=" * 72)
    print("Hello!")


    eliza_chatbot.modified_converse()






def demo():
    print("test")
    eliza_chat()






if __name__ == "__main__":
    demo()


test
Therapist
---------
Talk to the program by typing in plain English, using normal upper-
and lower-case letters and punctuation.  Enter "quit" when done.
Hello!


> hello


---Stemmed Input :  hello
---Total #of type:  1
---# of Tokens   :  1
Hello... What's your name?


> quit


---Stemmed Input :  quit
---Total #of type:  2
---# of Tokens   :  2
Let's change focus a bit... Tell me about your family.
